# Crawler for Euroleague Player Stats
This is as Crawler to fetch the Game Player Stats from the Euroleague Homepage.
It is supposed to safe them as a Parquet File in the end.
This Crawler is as complicated as it is, because the Website of the Euroleague relies heavily on Loading the Data through callbacks, even if links are provided. So an approach through the Selenium Web Interface was chosen. Because later on in the project it is the idea to use TensorFlow and with that utilizing the GPU, it is being developed on the WSL to easily use the GPU and not have to use fucking anaconda and also be future proof, as TensorFlow will not support CUDA through Windows after 2.10. Therefore, if this is supposed to be run through Windows the Import are parted based on the platform.

In [75]:
from bs4 import BeautifulSoup as bs
from datetime import datetime
from copy import copy
from os import path
import polars as pl
import seaborn as sns
import pendulum
import platform
import requests
import tqdm
import time
import re

driver = None

if platform.system() == "Windows":
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.by import By
    from webdriver_manager.firefox import GeckoDriverManager
    driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))
if platform.system() == "Linux":
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.common.by import By
    from webdriver_manager.chrome import ChromeDriverManager
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)




## Team Stats
An Excel with the Average Team Stats, even advanced Stats was available as an Excel and so it is imported here.

In [9]:
team_stats = path.join("..", "data", "external", "Team stats.xlsx")

In [10]:
stats_df = pl.read_excel(team_stats, sheet_name="stats_clean", engine="xlsx2csv")

Print for verification

In [11]:
print(stats_df)

shape: (19, 53)
┌──────┬──────────────────────────┬──────┬──────┬───┬─────────────┬──────────┬──────────┬──────────┐
│ RNK  ┆ TEAM                     ┆ GP   ┆ W    ┆ … ┆ DF 100 Poss ┆ BLK%     ┆ eFG%     ┆ TS%      │
│ ---  ┆ ---                      ┆ ---  ┆ ---  ┆   ┆ ---         ┆ ---      ┆ ---      ┆ ---      │
│ i64  ┆ str                      ┆ i64  ┆ i64  ┆   ┆ f64         ┆ f64      ┆ f64      ┆ f64      │
╞══════╪══════════════════════════╪══════╪══════╪═══╪═════════════╪══════════╪══════════╪══════════╡
│ 1    ┆ ALBA Berlin              ┆ 34   ┆ 14   ┆ … ┆ 28.45806    ┆ 5.530303 ┆ 0.527565 ┆ 0.560578 │
│ 2    ┆ Anadolu Efes Istanbul    ┆ 40   ┆ 23   ┆ … ┆ 27.650806   ┆ 7.477906 ┆ 0.558292 ┆ 0.596395 │
│ 3    ┆ AS Monaco                ┆ 38   ┆ 21   ┆ … ┆ 28.160772   ┆ 5.895357 ┆ 0.548507 ┆ 0.582909 │
│ 4    ┆ AX Armani Exchange Milan ┆ 36   ┆ 22   ┆ … ┆ 26.815057   ┆ 6.761269 ┆ 0.517345 ┆ 0.548064 │
│ 5    ┆ Bitci Baskonia           ┆ 32   ┆ 12   ┆ … ┆ 26.515783   ┆ 6.31068

## Getting the Game Links
To get the Game Links, the Game Center page is being crawled. So far the Function does not accept any inputs for the Season and is Hardcoded to the 2022 Season, because also no checks to until which game in the season has been played has not been implemented, but is a plan for the Future. To get the Game Links the we have to use the Selenium Webdriver to click through the pagination of each site. The "assert_round" Method is used to verify that the page has been loaded properly to only then start taking the links, because otherwise with the callback loading we might not end up with all the data we want.

In [12]:
def assert_round(driver, season, round):
    elm = driver.find_elements(By.CLASS_NAME, "game-center-select-shared_selectedValue__4xru_")
    corr_season = False
    corr_round = False
    for s in elm:
        if s.get_attribute("aria-labelledby") == "prefix_seasons_select":
            corr_season = s.text == season
        elif s.get_attribute("aria-labelledby") == "prefix_rounds_select":
            corr_round = s.text == round
    return corr_round & corr_season


def load_with_selenium():
    links = []
    # open browser and page
    url="C:\\Programs\\Python36\\BrowersDriver\\geckodriver.exe"
    # Accept cookies
    eu_url = "https://www.euroleaguebasketball.net/euroleague/"

    
    count = 1

    driver.get(eu_url)
    delay = 10
    element = WebDriverWait(driver, delay).until(lambda x: x.find_element(By.ID, "onetrust-accept-btn-handler"))
    element.click()

    driver.get("https://www.euroleaguebasketball.net/euroleague/game-center/")
    date_element = WebDriverWait(driver, delay).until(lambda x: x.find_element(By.CLASS_NAME, "game-center-group_groupTitle__5RCk4"))
    old_date = date_element.text

    for round in tqdm.tqdm(range(1, 35)):
        entry_url = f"https://www.euroleaguebasketball.net/euroleague/game-center/?round={round}&season=E2022"
        driver.get(entry_url)
        while not assert_round(driver, "2022-23", f"Round {round}"):
            time.sleep(1)
        
        while not old_date != driver.find_element(By.CLASS_NAME, "game-center-group_groupTitle__5RCk4").text:
            time.sleep(1)
        # s_element = WebDriverWait(driver, delay).until(lambda x: assert_round(x, "2022-23", f"Round {round}"))

        body = driver.find_element(By.TAG_NAME, "body")

        body_content = body.get_attribute("innerHTML")

        soup = bs(body_content, "html.parser")

        for link in soup.find_all("a"):
            links.append(link.get("href"))
            
    driver.close()
    link_format = r"/euroleague/game-center/2022-23/[\w-]+/E2022/[\d]{1,3}/"
    r = re.compile(link_format)
    newlist = list(filter(r.match, links))

    return list(dict.fromkeys(newlist))

In [13]:
res = load_with_selenium()

100%|██████████| 34/34 [00:55<00:00,  1.63s/it]


In [14]:
print(len(res))
print(res)

306
['/euroleague/game-center/2022-23/ldlc-asvel-villeurbanne-ea7-emporio-armani-milan/E2022/2/', '/euroleague/game-center/2022-23/panathinaikos-athens-real-madrid/E2022/1/', '/euroleague/game-center/2022-23/maccabi-playtika-tel-aviv-zalgiris-kaunas/E2022/3/', '/euroleague/game-center/2022-23/valencia-basket-cazoo-baskonia-vitoria-gasteiz/E2022/5/', '/euroleague/game-center/2022-23/fc-bayern-munich-fenerbahce-beko-istanbul/E2022/4/', '/euroleague/game-center/2022-23/anadolu-efes-istanbul-crvena-zvezda-meridianbet-belgrade/E2022/6/', '/euroleague/game-center/2022-23/alba-berlin-partizan-mozzart-bet-belgrade/E2022/7/', '/euroleague/game-center/2022-23/virtus-segafredo-bologna-as-monaco/E2022/9/', '/euroleague/game-center/2022-23/fc-barcelona-olympiacos-piraeus/E2022/8/', '/euroleague/game-center/2022-23/crvena-zvezda-meridianbet-belgrade-panathinaikos-athens/E2022/11/', '/euroleague/game-center/2022-23/as-monaco-anadolu-efes-istanbul/E2022/10/', '/euroleague/game-center/2022-23/virtus-se

## Getting the Stats from the individual players from the game
The get_game_data Function takes a game link partial, fetched by the previous function, to then scrape the table contents of the Boxscore.
### Taking the player names
As the Class names for the Players and the Stats differ, we first have to take the player names. These also provide links to the player pages, which are stored along the name for potential later use.
### Taking the Stat Data
The data is taken by column and each column first is represented as polars DataFrame where each column is a player. Because of irregularities in player counts, they might not alway be 12 + the team and total column, we seperate them on concatenation. Afterwards the data ist perpared.

In [128]:
def player_column_dicts(player_names: list, box1_columns: list, box2_columns: list, soup: bs) -> tuple[dict[str,str], dict[str,str]]:
    div_class = "game-box-scores-table-grouped-tab_tableGroupedRowsContainer__rmPnn game-box-scores-table-grid-sizing_tableGridSizing__KbFAw"
    name_div = soup.find_all("div", attrs={"class": div_class})
    
    # Get amount of player name divs and divs themselves for home and away
    home_team_players = name_div[0].find_all("div", recursive=False)[2:]
    home_team_player_count = len(home_team_players)
    away_team_players = name_div[1].find_all("div", recursive=False)[2:]
    away_team_player_count = len(away_team_players)
    
    # Check if second last item contains Team, if it does there are 2 extra rows, else only the total so one extra row
    # this is important for taking the appropriate amount of players from the list
    # TODO: Take player names here directly, like why the extra steps
    if home_team_players[-2].find("b").get_text().strip() == "Team":
        home_team_player_count -= 2
        home_team_team = True
    else:
        home_team_player_count -= 1
        home_team_team = False
    if away_team_players[-2].find("b").get_text().strip() == "Team":
        away_team_player_count -= 2
        away_team_team = True
    else:
        away_team_player_count -= 1
        away_team_team = False

    t1_names = player_names[:home_team_player_count]
    t2_names = player_names[home_team_player_count:]

    t1_renames = {}
    for i, x in enumerate(t1_names):
        t1_renames[f"column_{i}"] = x[1]
    if home_team_team:
        t1_renames[f"column_{i+1}"] = "Team"
        t1_renames[f"column_{i+2}"] = "Total"
    else:
        t1_renames[f"column_{i+1}"] = "Total"

    t2_renames = {}
    for j, x in enumerate(t2_names):
        t2_renames[f"column_{j}"] = x[1]
    if away_team_team:
        t2_renames[f"column_{j+1}"] = "Team"
        t2_renames[f"column_{j+2}"] = "Total"
    else:
        t2_renames[f"column_{j+1}"] = "Total"
    
    return t1_renames, t2_renames


def get_game_info(soup: bs) -> dict:
    """Takes a game page and returns relevant data like team names, Date and Time and Round

    Args:
        soup (bs): BeatifoulSoup4 Object with parsed html

    Returns:
        dict: dict with contents home_team, away_team, round, date, time
    """
    name_class = "club-info_name__xB4rz"
    team_names = []
    for element in soup.find_all("h1", attrs={"class": name_class}):
        team_names.append(element.get_text().strip())

    game_info_class = "event-info_listItem__Iu4ou"
    game_info = []
    for element in soup.find_all("li", attrs={"class": game_info_class}):
        game_info.append(element.get_text().strip())
    
    return {
        "home_team": team_names[0],
        "away_team": team_names[1],
        "round": re.search(r"(\d{1,2})", game_info[0])[0],
        "date": pendulum.from_format(game_info[1], "D MMM YYYY"),
        "time": game_info[3]
    }


def get_game_data(link: str) -> pl.DataFrame:
    """Pulls game data for game under specified link from euroleague homepage

    Args:
        link (str): euroleague page game link

    Returns:
        pl.DataFrame: DataFrame with all relevant Data From Site
    """
    
    base_url = f"https://www.euroleaguebasketball.net{link}"
    site = requests.get(base_url)

    # Open Parser 
    soup = bs(str(site.content), 'html.parser')
    # Class names for relevant Parts of HTML
    columns_class = "game-box-scores-table-grouped-column_tableGroupedColumnStatGroupContainer__Oci84"
    row_class = "game-box-scores-table-grouped-column_tableGroupedRowStatGroupContainer__wluIz"
    stat_name_class = "game-box-scores-table-grouped-column_groupedTableHeading__XTxR_"
    stat_cell_class = "game-box-scores-table-grouped-column_tableStatCell__4zJlJ"

    # Class for Player link, regex the name of the Player
    player_link = "game-box-scores-table-grouped-tab_playerLink__MqVrl"
    re_player = r"/euroleague/players/([\w-]+)/[\d\w]+/"
    players = []
    for link in soup.find_all("a", attrs={"class": player_link}):
        players.append((link.get("href"), re.match(re_player, link.get("href")).groups()[0]))

    # create a dataframe for the output witht the player names and links
    output_df = pl.DataFrame(data=players, schema={"links": str, "names": str})

    # Find all the Data Columns for a Team
    # Save them in List of Columns to be appended later
    stats_collection = []
    for element in soup.find_all("div", attrs={"class": columns_class}):
        col_stats = []
        col_heads = [stathead.text for stathead in element.find_all("div", attrs={"class": stat_name_class})]
        for row in element.find_all("div", attrs={"class": row_class}):
            stats = row.find_all("div", attrs={"class": stat_cell_class})
            if stats:
                col_stats.append([stat.text for stat in stats])
        schema = {}
        for i in col_heads:
            schema["i"]: str
        stats_df = pl.DataFrame(data=col_stats, schema=schema)
        stats_collection.append(stats_df)

    boxscore_t1 = pl.concat(stats_collection[:9])
    boxscore_t2 = pl.concat(stats_collection[9:])

    # TODO: Maybe put in seperate File for ease of reading
    stat_names = pl.Series(values=[
        "Minutes", "Points",
        "2P_Made/Attempted", "2P_Percentage",
        "3P_Made/Attempted", "3P_Percentage",
        "FTP_Made/Attempted", "FTP_Percentage",
        "O_Rebounds", "D_Rebounds", "Total_Rebounds",
        "Assists", "Steals", "Turnovers",
        "Blocks", "ReceivedBlocks",
        "FoulsCommited", "FoulReceived",
        "Effectivity", "PlusMinus"
    ], name="Stat")

    game_info = get_game_info(soup)

    home_names, away_names = player_column_dicts(players, boxscore_t1.columns, boxscore_t2.columns, soup)
    boxscore_t1 = (
        boxscore_t1
        .rename(home_names)  # add player names to as column names
        .with_columns(stat_names)  # add stat names to rows
        .transpose( # Pivot Boxscore to have player names as data Entry to Filter by
            include_header = True,
            header_name = "Player",
            column_names = "Stat"
        )
        .with_columns(
            pl.lit(game_info["home_team"]).alias("Team")
        )
    )
    boxscore_t2 = (
        boxscore_t2
        .rename(away_names)
        .with_columns(stat_names)
        .transpose(
            include_header = True,
            header_name = "Player",
            column_names = "Stat"
        )
        .with_columns(
            pl.lit(game_info["away_team"]).alias("Team")
        )
    )
    boxscore : pl.DataFrame = pl.concat([boxscore_t1, boxscore_t2])
    percent_format = r"([\d.]+)"
    # Do all the Type conversions to String, float, etc
    boxscore = (
        boxscore
        .filter(pl.col("Player").ne("Team"))
        .with_columns(
            pl.col("Minutes").str.extract_groups(r"(?P<Minutes>\d+):(?P<Seconds>\d+)"),
            pl.col("Points").cast(pl.Int16),
            pl.col("2P_Made/Attempted").str.extract_groups(r"(?P<TwoPMakes>\d{1,2})/(?P<TwoPAttempts>\d{1,2})"),
            pl.col("3P_Made/Attempted").str.extract_groups(r"(?P<ThreePMakes>\d{1,2})/(?P<ThreePAttempts>\d{1,2})"),
            pl.col("FTP_Made/Attempted").str.extract_groups(r"(?P<FTMakes>\d{1,2})/(?P<FTAttempts>\d{1,2})"),
            pl.col("2P_Percentage").str.extract(percent_format, 1).cast(pl.Float32).truediv(100),
            pl.col("3P_Percentage").str.extract(percent_format, 1).cast(pl.Float32).truediv(100),
            pl.col("FTP_Percentage").str.extract(percent_format, 1).cast(pl.Float32).truediv(100),
            pl.col("O_Rebounds").cast(pl.Int16),
            pl.col("D_Rebounds").cast(pl.Int16),
            pl.col("Total_Rebounds").cast(pl.Int16),
            pl.col('Assists').cast(pl.Int16), 
            pl.col('Steals').cast(pl.Int16),
            pl.col('Turnovers').cast(pl.Int16),
            pl.col('Blocks').cast(pl.Int16),
            pl.col('ReceivedBlocks').cast(pl.Int16),
            pl.col('FoulsCommited').cast(pl.Int16),
            pl.col('FoulReceived').cast(pl.Int16),
            pl.col('Effectivity').cast(pl.Int16),
            pl.when(pl.col("PlusMinus") == "").then(pl.lit("0")).otherwise(pl.col("PlusMinus")).cast(pl.Int16).alias("PlusMinus")
        )
        .unnest(["2P_Made/Attempted", "3P_Made/Attempted", "FTP_Made/Attempted", "Minutes"])
        .with_columns(
            Seconds=pl.col("Minutes").cast(pl.Int64).mul(60).add(pl.col("Seconds").cast(pl.Int64)),
            Date=pl.lit(game_info["date"]),
            Round=pl.lit(game_info["round"]),
            GameTime=pl.lit(game_info["time"])
        )
        .drop("Minutes")
    )

    return boxscore

data = get_game_data(res[0])
print(data.select(["Team", "Player", "Seconds", "Date", "GameTime"]))

shape: (26, 5)
┌──────────────────────────┬────────────────────┬─────────┬─────────────────────────┬──────────┐
│ Team                     ┆ Player             ┆ Seconds ┆ Date                    ┆ GameTime │
│ ---                      ┆ ---                ┆ ---     ┆ ---                     ┆ ---      │
│ str                      ┆ str                ┆ i64     ┆ datetime[μs, UTC]       ┆ str      │
╞══════════════════════════╪════════════════════╪═════════╪═════════════════════════╪══════════╡
│ LDLC ASVEL Villeurbanne  ┆ jonah-mathews      ┆ 1534    ┆ 2022-10-06 00:00:00 UTC ┆ 18:00    │
│ LDLC ASVEL Villeurbanne  ┆ anthony-polite     ┆ 341     ┆ 2022-10-06 00:00:00 UTC ┆ 18:00    │
│ LDLC ASVEL Villeurbanne  ┆ amine-noua         ┆ 1946    ┆ 2022-10-06 00:00:00 UTC ┆ 18:00    │
│ LDLC ASVEL Villeurbanne  ┆ paul-lacombe       ┆ 0       ┆ 2022-10-06 00:00:00 UTC ┆ 18:00    │
│ LDLC ASVEL Villeurbanne  ┆ antoine-diot       ┆ 403     ┆ 2022-10-06 00:00:00 UTC ┆ 18:00    │
│ …            

In [129]:
import time
boxscores = []
for link in tqdm.tqdm(res):
    boxscores.append(get_game_data(link))
    time.sleep(1.5)

100%|██████████| 306/306 [11:27<00:00,  2.25s/it]


In [132]:
all_stats: pl.DataFrame = pl.concat(boxscores).rechunk()

In [155]:
all_stats.shape
print(all_stats.schema)

stats_with_running_averages = (
    all_stats
    .sort("Date")
    .with_columns(
        pl.col("Round").cast(pl.Int16),
        pl.col("TwoPMakes").cast(pl.Int16),
        pl.col("TwoPAttempts").cast(pl.Int16),
        pl.col("ThreePMakes").cast(pl.Int16),
        pl.col("ThreePAttempts").cast(pl.Int16),
        pl.col("FTMakes").cast(pl.Int16),
        pl.col("FTAttempts").cast(pl.Int16),
        game_number=pl.col("Player").cum_count().over("Player")
    )
    .with_columns(
        avg_seconds=pl.col("Seconds").cum_sum().over("Player").truediv("game_number"),
        avg_points=pl.col('Points').cum_sum().over("Player").truediv("game_number"),
        avg_TwoPMakes = pl.col("TwoPMakes").cum_sum().over("Player").truediv("game_number"),
        avg_TwoPAttempts = pl.col("TwoPAttempts").cum_sum().over("Player").truediv("game_number"),
        avg_2P_Percentage = pl.col("2P_Percentage").cum_sum().over("Player").truediv("game_number"),
        avg_ThreePMakes = pl.col("ThreePMakes").cum_sum().over("Player").truediv("game_number"),
        avg_ThreePAttempts = pl.col("ThreePAttempts").cum_sum().over("Player").truediv("game_number"),
        avg_3P_Percentage = pl.col("3P_Percentage").cum_sum().over("Player").truediv("game_number"),
        avg_FTMakes = pl.col("FTMakes").cum_sum().over("Player").truediv("game_number"),
        avg_FTAttempts = pl.col("FTAttempts").cum_sum().over("Player").truediv("game_number"),
        avg_FTP_Percentage = pl.col("FTP_Percentage").cum_sum().over("Player").truediv("game_number"),
        avg_O_Rebounds = pl.col("O_Rebounds").cum_sum().over("Player").truediv("game_number"),
        avg_D_Rebounds = pl.col("D_Rebounds").cum_sum().over("Player").truediv("game_number"),
        avg_Total_Rebounds = pl.col("Total_Rebounds").cum_sum().over("Player").truediv("game_number"),
        avg_Assists = pl.col("Assists").cum_sum().over("Player").truediv("game_number"),
        avg_Steals = pl.col("Steals").cum_sum().over("Player").truediv("game_number"),
        avg_Turnovers = pl.col("Turnovers").cum_sum().over("Player").truediv("game_number"),
        avg_Blocks = pl.col("Blocks").cum_sum().over("Player").truediv("game_number"),
        avg_ReceivedBlocks = pl.col("ReceivedBlocks").cum_sum().over("Player").truediv("game_number"),
        avg_FoulsCommited = pl.col("FoulsCommited").cum_sum().over("Player").truediv("game_number"),
        avg_FoulReceived = pl.col("FoulReceived").cum_sum().over("Player").truediv("game_number"),
        avg_Effectivity = pl.col("Effectivity").cum_sum().over("Player").truediv("game_number"),
        avg_PlusMinus = pl.col("PlusMinus").cum_sum().over("Player").truediv("game_number")
    )
)
print(stats_with_running_averages.filter(pl.col("Player").eq("Total")).select("Player", "Seconds", "avg_seconds", "Round", "game_number"))

OrderedDict([('Player', String), ('Seconds', Int64), ('Points', Int16), ('TwoPMakes', String), ('TwoPAttempts', String), ('2P_Percentage', Float32), ('ThreePMakes', String), ('ThreePAttempts', String), ('3P_Percentage', Float32), ('FTMakes', String), ('FTAttempts', String), ('FTP_Percentage', Float32), ('O_Rebounds', Int16), ('D_Rebounds', Int16), ('Total_Rebounds', Int16), ('Assists', Int16), ('Steals', Int16), ('Turnovers', Int16), ('Blocks', Int16), ('ReceivedBlocks', Int16), ('FoulsCommited', Int16), ('FoulReceived', Int16), ('Effectivity', Int16), ('PlusMinus', Int16), ('Team', String), ('Date', Datetime(time_unit='us', time_zone='UTC')), ('Round', String), ('GameTime', String)])
shape: (612, 5)
┌────────┬─────────┬──────────────┬───────┬─────────────┐
│ Player ┆ Seconds ┆ avg_seconds  ┆ Round ┆ game_number │
│ ---    ┆ ---     ┆ ---          ┆ ---   ┆ ---         │
│ str    ┆ i64     ┆ f64          ┆ i16   ┆ u32         │
╞════════╪═════════╪══════════════╪═══════╪═════════════╡


In [158]:
for player in stats_with_running_averages.get_column("Player").unique():
    teams = stats_with_running_averages.filter(pl.col("Player").eq(player)).get_column("Team").unique()
    if len(teams) > 1:
        print(player, teams)

achille-polonara shape: (2,)
Series: 'Team' [str]
[
	"Anadolu Efes I…
	"Zalgiris Kauna…
]
isaiah-taylor shape: (2,)
Series: 'Team' [str]
[
	"Zalgiris Kauna…
	"Anadolu Efes I…
]
Total shape: (18,)
Series: 'Team' [str]
[
	"Fenerbahce Bek…
	"Valencia Baske…
	"Cazoo Baskonia…
	"Partizan Mozza…
	"FC Barcelona"
	"Zalgiris Kauna…
	"FC Bayern Muni…
	"Anadolu Efes I…
	"LDLC ASVEL Vil…
	"ALBA Berlin"
	"Real Madrid"
	"Panathinaikos …
	"Olympiacos Pir…
	"EA7 Emporio Ar…
	"Virtus Segafre…
	"Crvena Zvezda …
	"AS Monaco"
	"Maccabi Playti…
]


In [157]:
print( stats_with_running_averages.filter(pl.col("Player").eq("anthony-polite")).get_column("Team").unique())

shape: (1,)
Series: 'Team' [str]
[
	"LDLC ASVEL Vil…
]
